In [2]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import time
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [3]:
np.random.seed(1234)
tf.set_random_seed(1234)

In [26]:
class PhysicsInformedNN:
    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        self.lb = lb
        self.ub = ub

        self.x_u = X_u[:,0:1]
        self.t_u = X_u[:,1:2]
        self.u = u
        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]
        self.layers = layers
        self.nu = nu

        self.weights, self.biases = self.initilize_NN(layers)

        self.sess = tf.Session()

        self.x_u_tf = tf.placeholder(tf.float32, shape=[None, self.x_u.shape[1]])
        self.t_u_tf = tf.placeholder(tf.float32, shape=[None, self.t_u.shape[1]])
        self.u_tf   = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])

        self.u_pred = self.net_u(self.x_u_tf, self.t_u_tf)
        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)

        self.loss = tf.reduce_mean(tf.square(self.u_pred - self.u_tf)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss,
                                                                method = 'L-BFGS-B',
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        init = tf.global_variables_initializer()
        self.sess.run(init)
    # Xavier初始化
    def initilize_NN(self, layers):
        weights = []
        biases  = []
        num_layers = len(layers)
        for l in range(0, num_layers - 1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    # 定义神经网络
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0, num_layers - 2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    # 用神经网络表示要求解的解函数u
    def net_u(self, x, t):
        u = self.neural_net(tf.concat([x,t], 1), self.weights, self.biases)
        return u
    # residual f
    def net_f(self, x, t):
        u = self.net_u(x, t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        f = u_t + u*u_x - self.nu * u_xx
        return f

    def train(self):
        tf_dict = {self.x_u_tf: self.x_u, self.t_u_tf: self.t_u, self.u_tf: self.u,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}
        self.optimizer.minimize(
                self.sess,
                feed_dict = tf_dict,
                fetches = [self.loss],
                loss_callback = self.callback
        )

    def callback(self, loss):
        print('Loss:', loss)

    def predict(self, X_star):
        u_star = self.sess.run(self.u_pred, {self.x_u_tf: X_star[:, 0:1], self.t_u_tf:X_star[:, 1:2]})
        f_star = self.sess.run(self.f_pred, {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]})
        return u_star, f_star

In [4]:
nu = 0.01/np.pi
noise = 0.0

N_u = 100
N_f = 10000
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

data = scipy.io.loadmat('./Burgers/burgers_shock.mat')

In [6]:
x = data['x'].flatten()[:, None]
t = data['t'].flatten()[:, None]
Exact =np.real(data['usol']).T

In [8]:
X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()[:, None]

In [14]:
# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)

In [15]:
# 初始点
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
# x=-1的边界点
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
# x=1的边界点
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

In [16]:
X_u_train = np.vstack([xx1, xx2, xx3]) #X_u_train.shape=(456, 2)
u_train = np.vstack([uu1, uu2, uu3])
X_f_train = lb + (ub-lb)*lhs(2, N_f) # pyODE.lhs 拉丁超立方体抽样
X_f_train = np.vstack((X_f_train, X_u_train))

In [17]:
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False) # 抽取N_u个点

In [18]:
X_u_train = X_u_train[idx]
u_train = u_train[idx]

In [27]:
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub, nu)

In [28]:
start_time = time.time()
model.train()
elapsed = time.time() - start_time
print('Training time: %.4f' % (elapsed))

Loss: 0.42414525
Loss: 0.5563602
Loss: 0.46767688
Loss: 0.31073987
Loss: 0.28346494
Loss: 0.23721446
Loss: 0.2302183
Loss: 0.22917107
Loss: 0.22861281
Loss: 0.22678992
Loss: 0.2238854
Loss: 0.21835928
Loss: 0.21594156
Loss: 0.21247709
Loss: 0.20967501
Loss: 0.20753984
Loss: 0.20662248
Loss: 0.19795725
Loss: 0.2631862
Loss: 0.1915716
Loss: 0.20574386
Loss: 0.18016978
Loss: 0.17123145
Loss: 0.16585656
Loss: 0.15939233
Loss: 0.13857876
Loss: 0.1290558
Loss: 0.12371823
Loss: 0.12080839
Loss: 0.11976642
Loss: 0.11768535
Loss: 0.11429052
Loss: 0.112101994
Loss: 0.111502916
Loss: 0.11097995
Loss: 0.110128686
Loss: 0.10773171
Loss: 0.10463181
Loss: 0.10391997
Loss: 0.1034346
Loss: 0.10330404
Loss: 0.103116564
Loss: 0.10243783
Loss: 0.10100962
Loss: 0.098729044
Loss: 0.09794581
Loss: 0.09808815
Loss: 0.09746608
Loss: 0.09722851
Loss: 0.096933275
Loss: 0.09621835
Loss: 0.09542988
Loss: 0.09393971
Loss: 0.0922682
Loss: 0.091253564
Loss: 0.09074334
Loss: 0.090428285
Loss: 0.089890525
Loss: 0.08897

In [29]:
u_pred, f_pred = model.predict(X_star)
# relative 2 norm
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
print('Error u: %e' % (error_u))
U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
Error = np.abs(Exact - U_pred)

Error u: 8.391375e-03
